In [83]:
from pynq import Overlay
from pynq import MMIO

# Program bitstream to FPGA
ol = Overlay('./impl_pynq_wrapper.bit')

In [84]:
def binary_to_float(binary_str):
    for i in range(16):
#         print(binary_str[i])
        if binary_str[i] not in ('0', '1'):
            if (binary_str[i] == 'b'):
                binary_str = binary_str[i+1:]
#                 print("trig")
                break
            else:
                print(f"Character at index {i} is neither '0' nor '1': {binary_str[i]}")
                return 0
    
    if (len(binary_str) < 16):
        binary_str = binary_str.zfill(16)
    elif (len(binary_str) > 16):
        binary_str = binary_str[-16:]
#     print(binary_str)


    """Convert a 32-bit binary string to a float."""
    sign = int(binary_str[0])
    integer_part = int(binary_str[1:5], 2)
    fractional_part = binary_str[5:]
    frac_value = 0.0
    for i, bit in enumerate(fractional_part):
        if bit == '1':
            frac_value += 1 / (2 ** (i + 1))
    float_value = integer_part + frac_value
    if sign == 1:
        float_value = -float_value
    return float_value

# print(binary_to_float("0b111111111110101"))

def float_to_custom_binary(float_value):
    # Determine the sign bit
    sign_bit = 0 if float_value >= 0 else 1

    # Get the absolute value of the float for further processing
    abs_value = abs(float_value)

    # Separate the integer and fractional parts
    integer_part = int(abs_value)
    fractional_part = abs_value - integer_part

    # Convert the integer part to binary (4 bits)
    integer_binary = format(integer_part & 0b1111, '04b')

    # Convert the fractional part to binary (27 bits)
    fractional_binary = ''
    for _ in range(11):
        fractional_part *= 2
        bit = int(fractional_part)
        fractional_binary += str(bit)
        fractional_part -= bit

    # Combine all parts into a single binary string
    custom_binary = f"0b{sign_bit}{integer_binary}{fractional_binary}"
    binary_value = int(custom_binary, 2)
    
    return binary_value

In [85]:
import numpy as np

# Define the values as a 2D NumPy array
values = np.array([
    [0.52158273, 0.24820144, 0.79676259, 0.55035971, 0.52697842, 0.16366906, 0.16546763, 0.73920863, 0.46402878, 0.34352518],
    [0.55555556, 0.46296296, 0.41077441, 0.42760943, 0.33501684, 0.57575758, 0.33164983, 0.52356902, 0.44949495, 0.42760943],
    [0.73218673, 0.75675676, 0.74692875, 0.72727273, 0.71007371, 0.70761671, 0.69041769, 0.71498771, 0.72481572, 0.75429975],
    [0.41613924, 0.35601266, 0.36392405, 0.3971519,  0.3528481,  0.36234177, 0.38132911, 0.27531646, 0.54588608, 0.40981013],
    [0.46638655, 0.79621849, 0.36344538, 0.71848739, 0.41806723, 0.3802521,  0.3802521,  0.59033613, 0.57352941, 0.60504202],
    [0.7002457,  0.66093366, 0.66830467, 0.74447174, 0.75184275, 0.68550369, 0.6953317,  0.7002457,  0.71007371, 0.65847666],
    [0.54871795, 0.34358974, 1.0,        0.74102564, 0.72564103, 0.46923077, 0.94871795, 0.74102564, 0.73333333, 0.46410256],
    [0.54411765, 0.48109244, 0.39705882, 0.57563025, 0.22268908, 0.22478992, 0.91176471, 0.41806723, 0.71428571, 0.44327731],
    [0.78343949, 0.78343949, 0.69002123, 0.68577495, 0.74309979, 0.75159236, 0.73460722, 0.66878981, 0.74734607, 0.78768577],
    [0.75776398, 0.73913043, 0.72049689, 0.77329193, 0.76397516, 0.7484472,  0.72981366, 0.7484472,  0.73602484, 0.75776398],
    [0.18642351, 0.26342452, 0.23100304, 0.22087133, 0.20364742, 0.32826748, 0.20263425, 0.26342452, 0.22695035, 0.19756839],
    [0.37025316, 0.38449367, 0.13765823, 0.13765823, 0.53639241, 0.42088608, 0.16297468, 0.10126582, 0.44303797, 0.47310127],
    [0.74927954, 0.74639769, 0.74063401, 0.74351585, 0.77233429, 0.74063401, 0.75216138, 0.73198847, 0.72910663, 0.74063401],
    [0.61663286, 0.62677485, 0.61866126, 0.62677485, 0.62271805, 0.62474645, 0.64908722, 0.55578093, 0.67951318, 0.65517241],
    [0.37668919, 0.6875,     0.36824324, 0.63851351, 0.47635135, 0.43918919, 0.47972973, 0.58952703, 0.54898649, 0.55574324],
    [0.7173913,  0.70496894, 0.71118012, 0.69254658, 0.72049689, 0.72360248, 0.71118012, 0.69254658, 0.70496894, 0.70807453],
    [0.77616279, 0.61337209, 0.59302326, 0.60174419, 0.59011628, 0.59593023, 0.38953488, 0.7994186,  0.55232558, 0.64534884],
    [0.72542373, 0.73898305, 0.71355932, 0.6779661,  0.68305085, 0.71864407, 0.73389831, 0.7,        0.67118644, 0.68135593],
    [0.34810127, 0.40031646, 0.37341772, 0.35601266, 0.36867089, 0.46044304, 0.42088608, 0.52689873, 0.34018987, 0.3528481 ],
    [0.6953317,  0.6977887,  0.72727273, 0.73710074, 0.74692875, 0.71253071, 0.69041769, 0.68796069, 0.6953317,  0.71498771],
    [0.61038961, 0.59253247, 0.61688312, 0.61850649, 0.58441558, 0.61525974, 0.60227273, 0.90097403, 0.46266234, 0.67857143],
    [0.6879562,  0.68430657, 0.80291971, 0.85218978, 0.82664234, 0.89416058, 0.85218978, 0.72627737, 0.79562044, 0.79379562],
    [0.41582492, 0.41077441, 0.55387205, 0.45791246, 0.44781145, 0.50841751, 0.59427609, 0.63299663, 0.44444444, 0.44781145],
    [0.7483871,  0.73870968, 0.67419355, 0.79677419, 0.71612903, 0.77096774, 0.8,        0.79032258, 0.75806452, 0.7483871 ],
    [0.76080692, 0.75504323, 0.75504323, 0.77809798, 0.76945245, 0.76080692, 0.76657061, 0.78962536, 0.78386167, 0.74639769],
    [0.57627119, 0.55932203, 0.51412429, 0.51694915, 0.52259887, 0.50847458, 0.52542373, 0.52542373, 0.56497175, 0.46045198],
    [0.55367232, 0.56214689, 0.57062147, 0.58474576, 0.57062147, 0.55367232, 0.53107345, 0.51977401, 0.52259887, 0.53954802],
    [0.33227848, 0.47151899, 0.38449367, 0.48575949, 0.33702532, 0.33227848, 0.31487342, 0.33544304, 0.39240506, 0.4335443 ],
    [0.63081395, 0.60465116, 0.61046512, 0.5872093,  0.61046512, 0.55232558, 0.6627907,  0.62209302, 0.41569767, 0.7994186 ],
    [0.73602484, 0.7515528,  0.76397516, 0.73913043, 0.72981366, 0.74534161, 0.76708075, 0.73913043, 0.71118012, 0.72049689],
    [0.76340111, 0.36044362, 0.33456562, 0.42513863, 0.38632163, 0.55268022, 0.56931608, 0.64140481, 0.3844732,  0.90018484],
    [0.19763514, 0.39864865, 0.22804054, 0.53547297, 0.49662162, 0.27195946, 0.4222973,  0.47635135, 0.3125,     0.39189189],
    [0.60521042, 0.65330661, 0.68937876, 0.71142285, 0.66933868, 0.66933868, 0.67134269, 0.63927856, 0.59719439, 0.63527054],
    [0.82066869, 0.81458967, 0.82066869, 0.82978723, 0.85714286, 0.87537994, 0.86930091, 0.84802432, 0.87234043, 0.89057751],
    [0.6753507,  0.70140281, 0.7755511,  0.68537074, 0.68537074, 0.74549098, 0.86573146, 0.73547094, 0.72344689, 0.77154309],
    [0.72360248, 0.7484472,  0.76086957, 0.74223602, 0.72360248, 0.73291925, 0.73602484, 0.75776398, 0.76708075, 0.72981366],
    [0.61411765, 0.18352941, 0.31529412, 0.70117647, 0.53176471, 0.68235294, 0.56235294, 0.61882353, 0.55529412, 0.62588235],
    [0.61016949, 0.56214689, 0.5480226,  0.51412429, 0.51129944, 0.51694915, 0.53672316, 0.55649718, 0.57627119, 0.60169492],
    [0.33641405, 0.34195933, 0.75970425, 0.29205176, 0.72828096, 0.32532348, 0.72643253, 0.37338262, 0.69316081, 0.34750462],
    [0.5990099,  0.60148515, 0.59158416, 0.58168317, 0.59158416, 0.61138614, 0.58663366, 0.58415842, 0.58168317, 0.59653465],
    [0.78115502, 0.76899696, 0.75379939, 0.79027356, 0.78723404, 0.78115502, 0.76291793, 0.78723404, 0.7781155,  0.78115502],
    [0.71142285, 0.74148297, 0.75150301, 0.74148297, 0.67735471, 0.78156313, 0.8256513,  0.76553106, 0.62925852, 0.63126253],
    [0.71342685, 0.75751503, 0.76553106, 0.68136273, 0.61122244, 0.69739479, 0.78557114, 0.80761523, 0.72745491, 0.74348697],
    [0.36603774, 0.90377358, 0.36792453, 0.89433962, 0.36415094, 0.90754717, 0.36792453, 0.44150943, 0.44339623, 0.61698113],
    [0.69487179, 0.71025641, 0.71282051, 0.7,        0.70512821, 0.71538462, 0.69487179, 0.70512821, 0.68461538, 0.68205128],
    [0.90732759, 0.89224138, 0.92456897, 0.92025862, 0.89655172, 0.93318966, 0.89655172, 0.91163793, 0.89008621, 0.90732759],
    [0.75373134, 0.76119403, 0.7960199,  0.83333333, 0.83084577, 0.78358209, 0.76368159, 0.79353234, 0.82587065, 0.82587065],
    [0.97966728, 0.78743068, 0.34935305, 0.31608133, 0.92606285, 0.80036969, 0.37707948, 0.33826248, 0.95563771, 0.7689464 ],
    [0.60380952, 0.61333333, 0.6,        0.58666667, 0.59619048, 0.60761905, 0.58095238, 0.5752381,  0.59238095, 0.58095238],
    [0.41694915, 0.38305085, 0.31355932, 0.47966102, 0.48813559, 0.51186441, 0.3,        0.51355932, 0.27966102, 0.54067797],
    [0.88865546, 0.48529412, 0.72268908, 0.46008403, 0.56932773, 0.37815126, 0.67226891, 0.45378151, 0.61554622, 0.45168067],
    [0.21790541, 0.41722973, 0.42905405, 0.42398649, 0.25168919, 0.72466216, 0.25337838, 0.46452703, 0.42567568, 0.375],
    [0.7960199, 0.76119403, 0.77363184, 0.76616915, 0.8159204, 0.80845771, 0.83333333, 0.82338308, 0.76119403, 0.7761194],
    [0.47058824, 0.52521008, 0.20378151, 0.20378151, 0.55042017, 0.4012605, 0.5987395, 0.4012605, 0.5987395, 0.43277311],
    [0.31864407, 0.47966102, 0.43559322, 0.63898305, 0.3440678, 0.3440678, 0.33389831, 0.36610169, 0.46610169, 0.37288136],
    [0.57176471, 0.61647059, 0.57176471, 0.57176471, 0.56470588, 0.58117647, 0.57411765, 0.58117647, 0.57176471, 0.57176471],
    [0.72981366, 0.7173913, 0.71118012, 0.69565217, 0.70807453, 0.70186335, 0.72360248, 0.73602484, 0.70186335, 0.69565217],
    [0.57425743, 0.58415842, 0.57673267, 0.59653465, 0.5990099, 0.58910891, 0.56683168, 0.57673267, 0.58910891, 0.58910891],
    [0.73291925, 0.73913043, 0.7515528, 0.75776398, 0.75776398, 0.74223602, 0.73602484, 0.74534161, 0.77329193, 0.7515528],
    [0.72399151, 0.83014862, 0.71549894, 0.69426752, 0.77494692, 0.75583864, 0.67515924, 0.63269639, 0.72186837, 0.7133758],
    [0.22972973, 0.21283784, 0.45608108, 0.26013514, 0.3277027, 0.36993243, 0.36655405, 0.41891892, 0.28040541, 0.26689189],
    [0.51495017, 0.25913621, 0.54152824, 0.25249169, 0.53156146, 0.24031008, 0.53156146, 0.26356589, 0.54263566, 0.24806202],
    [0.65605096, 0.67303609, 0.69214437, 0.67728238, 0.72399151, 0.88747346, 0.80467091, 0.79193206, 0.75583864, 0.72186837],
    [0.4040404, 0.47474747, 0.53198653, 0.53198653, 0.52861953, 0.55555556, 0.41919192, 0.46296296, 0.39225589, 0.4983165],
    [0.70761671, 0.70515971, 0.68796069, 0.74692875, 0.76412776, 0.68304668, 0.66584767, 0.7002457, 0.72235872, 0.71990172],
    [0.71935484, 0.71935484, 0.69677419, 0.73225806, 0.77741935, 0.78709677, 0.7516129, 0.73225806, 0.73548387, 0.74193548],
    [0.75216138, 0.73487032, 0.76945245, 0.74351585, 0.76080692, 0.77521614, 0.76080692, 0.76080692, 0.78097983, 0.77233429],
    [0.57116451, 0.62292052, 0.63770795, 0.60073937, 0.56931608, 0.5767098, 0.59149723, 0.62661738, 0.66543438, 0.66543438],
    [0.78957916, 0.7254509, 0.65931864, 0.79158317, 0.76753507, 0.70140281, 0.76352705, 0.749499, 0.78557114, 0.71142285],
    [0.77124183, 0.76034858, 0.81045752, 0.81917211, 0.79956427, 0.78649237, 0.77124183, 0.76688453, 0.78867102, 0.85185185],
    [0.23708207, 0.22796353, 0.18439716, 0.26342452, 0.20263425, 0.21175279, 0.21783181, 0.2147923, 0.25937183, 0.22593718],
    [0.52129817, 0.32048682, 0.30831643, 0.34482759, 0.3346856, 0.29614604, 0.45436105, 0.36105477, 0.37728195, 0.43610548],
    [0.52259887, 0.56497175, 0.5480226, 0.58474576, 0.57344633, 0.57344633, 0.51412429, 0.52259887, 0.50564972, 0.52542373],
    [0.80748663, 0.81550802, 0.86363636, 0.86363636, 0.86096257, 0.79144385, 0.76203209, 0.7459893, 0.77540107, 0.83957219],
    [0.71118012, 0.72670807, 0.7173913, 0.74223602, 0.73291925, 0.73291925, 0.70807453, 0.7484472, 0.73913043, 0.72981366],
    [0.8453159, 0.8540305, 0.78867102, 0.80392157, 0.79084967, 0.79302832, 0.81699346, 0.85185185, 0.83224401, 0.77342048],
    [0.78723404, 0.81458967, 0.79635258, 0.80243161, 0.81155015, 0.83282675, 0.7993921, 0.80547112, 0.81155015, 0.81155015],
    [0.60377358, 0.6, 0.60566038, 0.61132075, 0.59622642, 0.58867925, 0.61320755, 0.61132075, 0.59433962, 0.61320755],
    [0.57647059, 0.57882353, 0.59764706, 0.58352941, 0.56941176, 0.56, 0.58117647, 0.58117647, 0.58352941, 0.40705882],
    [0.20567376, 0.26443769, 0.22695035, 0.2077001, 0.14488349, 0.14488349, 0.2289767, 0.18439716, 0.17933131, 0.30800405]
])

boolean_values = np.array([
    True, False, False, True, True, False, True, True, False, False,
    True, True, False, False, True, False, True, False, False, False,
    False, False, False, False, False, False, False, False, True, False,
    True, True, False, False, False, False, True, False, True, False,
    False, False, False, True, False, False, False, True, False, False,
    True, True, False, True, False, False, False, False, False, False,
    True, True, False, False, False, False, False, False, False, False,
    True, False, False, False, False, False, False, False, False, True
])

In [86]:
# Reset
ol.arrhythmia_ip_0.write(0,1)
y1 = ol.arrhythmia_ip_0.read(44)
y2 = ol.arrhythmia_ip_0.read(48)
print("actual label:",boolean_values[sample_index])
print(binary_to_float(bin(y1)), binary_to_float(bin(y2)))
print("prediction:", binary_to_float(bin(y1)) > binary_to_float(bin(y2)))

actual label: True
0.0 0.0
prediction: False


In [87]:
# Run
wrong_count = 0
total_count = 0
for i in range(len(values)):
    ol.arrhythmia_ip_0.write(0,1)
    ol.arrhythmia_ip_0.write(0,0)
    sample_index = i
    ol.arrhythmia_ip_0.write(4,float_to_custom_binary(values[sample_index][9]))
    ol.arrhythmia_ip_0.write(8,float_to_custom_binary(values[sample_index][8]))
    ol.arrhythmia_ip_0.write(12,float_to_custom_binary(values[sample_index][7]))
    ol.arrhythmia_ip_0.write(16,float_to_custom_binary(values[sample_index][6]))
    ol.arrhythmia_ip_0.write(20,float_to_custom_binary(values[sample_index][5]))
    ol.arrhythmia_ip_0.write(24,float_to_custom_binary(values[sample_index][4]))
    ol.arrhythmia_ip_0.write(28,float_to_custom_binary(values[sample_index][3]))
    ol.arrhythmia_ip_0.write(32,float_to_custom_binary(values[sample_index][2]))
    ol.arrhythmia_ip_0.write(36,float_to_custom_binary(values[sample_index][1]))
    ol.arrhythmia_ip_0.write(40,float_to_custom_binary(values[sample_index][0]))
    y1 = ol.arrhythmia_ip_0.read(44)
    y2 = ol.arrhythmia_ip_0.read(48)
    actual_label = boolean_values[sample_index]
    prediction = binary_to_float(bin(y1)) > binary_to_float(bin(y2))
    if (actual_label != prediction):
        wrong_count += 1
    if (prediction == True):
        true_count += 1
    total_count += 1

print("total_count=", total_count)
print("wrong_count=", wrong_count)
print("accuracy=",((total_count - wrong_count)/total_count)*100)

total_count= 80
wrong_count= 23
accuracy= 71.25
